<a href="https://colab.research.google.com/github/github-pooja/Scraped-ML-Projects-FlipRobo/blob/main/Image%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import os
Test_dir = 'https://drive.google.com/drive/folders/1PQHP8h6FFWBJReb1le2U6dLSDUHshg9x?usp=sharing'
Train_dir = 'https://drive.google.com/drive/folders/1XYrTXOygSOY4byJP70UPaivpnRXPT-26?usp=sharing'


In [12]:
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

import cv2
import os

import numpy as np

In [13]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()

In [16]:
#Loading the Data
labels = ['Sarees', 'Jeans','Trousers']
img_size = 224
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [20]:
!pip install PyDrive

In [21]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [33]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [38]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

MIMETYPES = {
        # Drive Document files as MS dox
        'application/vnd.google-apps.document': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
        # Drive Sheets files as MS Excel files.
        'application/vnd.google-apps.spreadsheet': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
        # Drive presentation as MS pptx
        'application/vnd.google-apps.presentation': 'application/vnd.openxmlformats-officedocument.presentationml.presentation'
        # see https://developers.google.com/drive/v3/web/mime-types
    }
EXTENSTIONS = {
        'application/vnd.google-apps.document': '.docx',
        'application/vnd.google-apps.spreadsheet': '.xlsx',
        'application/vnd.google-apps.presentation': '.pptx'
}

f = open("failed.txt","w+")
folder_id = '1fI_AmhrqK_08HNqVsTOIA6A3PpvaZJbh'
root = 'drive_download'
os.mkdir(root)

def escape_fname(name):
    return name.replace('/','_')

def search_folder(folder_id, root):
    file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % folder_id}).GetList()
    for file in file_list:
        # print('title: %s, id: %s, kind: %s' % (file['title'], file['id'], file['mimeType']))
        # print(file)
        if file['mimeType'].split('.')[-1] == 'folder':
            foldername = escape_fname(file['title'])
            create_folder(root,foldername)
            search_folder(file['id'], '{}{}/'.format(root,foldername))
        else:
            download_mimetype = None
            filename = escape_fname(file['title'])
            filename = '{}{}'.format(root,filename)
            try:
                print('DOWNLOADING:', filename)
                if file['mimeType'] in MIMETYPES:
                    download_mimetype = MIMETYPES[file['mimeType']]

                    file.GetContentFile(filename+EXTENSTIONS[file['mimeType']], mimetype=download_mimetype)
                else:
                    file.GetContentFile(filename)
            except:
                print('FAILED')
                f.write(filename+'\n')

def create_folder(path,name):
    os.mkdir('{}{}'.format(path,escape_fname(name)))

search_folder(folder_id,root+'/')
f.close() 

DOWNLOADING: drive_download/Jeans52.jpg
DOWNLOADING: drive_download/Jeans53.jpg
DOWNLOADING: drive_download/Jeans50.jpg
DOWNLOADING: drive_download/Jeans51.jpg
DOWNLOADING: drive_download/Jeans49.jpg
DOWNLOADING: drive_download/Jeans48.jpg
DOWNLOADING: drive_download/Jeans47.jpg
DOWNLOADING: drive_download/Jeans45.jpg
DOWNLOADING: drive_download/Jeans46.jpg
DOWNLOADING: drive_download/Jeans44.jpg
DOWNLOADING: drive_download/Jeans43.jpg
DOWNLOADING: drive_download/Jeans41.jpg
DOWNLOADING: drive_download/Jeans40.jpg
DOWNLOADING: drive_download/Jeans42.jpg
DOWNLOADING: drive_download/Trousers199.jpg
DOWNLOADING: drive_download/Trousers198.jpg
DOWNLOADING: drive_download/Trousers196.jpg
DOWNLOADING: drive_download/Trousers197.jpg
DOWNLOADING: drive_download/Trousers195.jpg
DOWNLOADING: drive_download/Trousers194.jpg
DOWNLOADING: drive_download/Trousers193.jpg
DOWNLOADING: drive_download/Trousers192.jpg
DOWNLOADING: drive_download/Trousers191.jpg
DOWNLOADING: drive_download/Trousers190.jpg


In [52]:
train='/content/drive_download/'

In [59]:
from PIL import Image
import os
import numpy as np
import re
def get_data(train):
    all_images_as_array=[]
    label=[]
    for filename in os.listdir(train):
        try:
            if re.match(r'Jeans',filename):
                  label.append(1)
            if re.match(r'Trousers',filename):
                  label.append(2)
            else:
                label.append(0)
            img=Image.open(path + filename)
            np_array = np.asarray(img)
            l,b,c = np_array.shape
            np_array = np_array.reshape(l*b*c,)
            all_images_as_array.append(np_array)
        except:
            continue
    return np.array(all_images_as_array), np.array(label)
path_to_train_set = '/content/Test'
path_to_test_set = '/content/Train'
X_train,y_train = get_data(train)
X_test, y_test = get_data(train)
print('X_train set : ',X_train)
print('y_train set : ',y_train)
print('X_test set : ',X_test)
print('y_test set : ',y_test)

X_train set :  []
y_train set :  [2 0 0 0 0 0 2 0 2 0 2 2 2 0 2 1 0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 0 0 2 0 0 0
 1 0 1 0 0 1 0 0 0 0 0 2 2 2 2 2 1 0 1 0 1 0 1 0 2 2 0 2 0 1 0 2 0 0 1 0 2
 0 0 2 1 0 0 1 0 0 0 2 0 1 0 0 2 0 0 0 1 0 0 2 2 2 0 2 2 2 0 0 1 0 2 2 2 2
 2 1 0 2 2 0 1 0 1 0 0 1 0 1 0 2 2 1 0 2 2 2 0 2 0 1 0 2 0 1 0 1 0 2 1 0 1
 0 2 1 0 2 2 1 0 1 0 1 0 2 2 1 0 0 0 2 2 2 1 0 0 0 2 1 0 1 0 2 2 2 1 0 1 0
 2 2 2 0 1 0 0 1 0 1 0 2 2 1 0 0 0 2 0 1 0 1 0 2 0 0 2 2 2 2 0 1 0 0 0 0 1
 0 1 0 1 0 1 0 0 0 0 1 0 2 0 1 0 2 1 0 2 0 1 0 2 0 2 0 2 0 1 0 0 1 0 0 1 0
 0 2 2 0 0 2 2 1 0 1 0 0 2 2 2 0 1 0 2 1 0 0 0 1 0 2 1 0 0 2 2 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 2 2 2 1 0 0 0 1 0 0 0 2 0 1 0 2 1 0 1 0 2 2 0
 2 2 2 0 1 0 1 0 2 2 2 0 2 0 1 0 1 0 1 0 0 1 0 0 1 0 1 0 1 0 2 1 0 2 0 0 0
 0 0 1 0 2 2 1 0 0 1 0 0 0 1 0 1 0 2 0 2 1 0 0 2 1 0 0 2 1 0 0 1 0 0 2 2 0
 1 0 0 2 0 1 0 2 2 1 0 1 0 1 0 2 2 2 0 1 0 0 1 0 2 2 1 0 2 0 1 0 1 0 0 2 0
 0 1 0 2 0 0 0 0 1 0 0 1 0 0 0 2 2 2 1 0 1 0 0 2 0 1 0 0 1 0 0 0 0 

In [60]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

MIMETYPES = {
        # Drive Document files as MS dox
        'application/vnd.google-apps.document': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
        # Drive Sheets files as MS Excel files.
        'application/vnd.google-apps.spreadsheet': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
        # Drive presentation as MS pptx
        'application/vnd.google-apps.presentation': 'application/vnd.openxmlformats-officedocument.presentationml.presentation'
        # see https://developers.google.com/drive/v3/web/mime-types
    }
EXTENSTIONS = {
        'application/vnd.google-apps.document': '.docx',
        'application/vnd.google-apps.spreadsheet': '.xlsx',
        'application/vnd.google-apps.presentation': '.pptx'
}

f = open("failed.txt","w+")
folder_id = '16iRalP0f2DUFXX1954FA-LzaSxH5i8mQ'
root = 'drive_download_Test'
os.mkdir(root)

def escape_fname(name):
    return name.replace('/','_')

def search_folder(folder_id, root):
    file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % folder_id}).GetList()
    for file in file_list:
        # print('title: %s, id: %s, kind: %s' % (file['title'], file['id'], file['mimeType']))
        # print(file)
        if file['mimeType'].split('.')[-1] == 'folder':
            foldername = escape_fname(file['title'])
            create_folder(root,foldername)
            search_folder(file['id'], '{}{}/'.format(root,foldername))
        else:
            download_mimetype = None
            filename = escape_fname(file['title'])
            filename = '{}{}'.format(root,filename)
            try:
                print('DOWNLOADING:', filename)
                if file['mimeType'] in MIMETYPES:
                    download_mimetype = MIMETYPES[file['mimeType']]

                    file.GetContentFile(filename+EXTENSTIONS[file['mimeType']], mimetype=download_mimetype)
                else:
                    file.GetContentFile(filename)
            except:
                print('FAILED')
                f.write(filename+'\n')

def create_folder(path,name):
    os.mkdir('{}{}'.format(path,escape_fname(name)))

search_folder(folder_id,root+'/')
f.close() 

DOWNLOADING: drive_download_Test/Jeans39.jpg
DOWNLOADING: drive_download_Test/Jeans38.jpg
DOWNLOADING: drive_download_Test/Jeans36.jpg
DOWNLOADING: drive_download_Test/Jeans37.jpg
DOWNLOADING: drive_download_Test/Jeans34.jpg
DOWNLOADING: drive_download_Test/Jeans35.jpg
DOWNLOADING: drive_download_Test/Jeans32.jpg
DOWNLOADING: drive_download_Test/Jeans33.jpg
DOWNLOADING: drive_download_Test/Jeans30.jpg
DOWNLOADING: drive_download_Test/Jeans31.jpg
DOWNLOADING: drive_download_Test/Jeans29.jpg
DOWNLOADING: drive_download_Test/Jeans28.jpg
DOWNLOADING: drive_download_Test/Jeans27.jpg
DOWNLOADING: drive_download_Test/Jeans25.jpg
DOWNLOADING: drive_download_Test/Jeans26.jpg
DOWNLOADING: drive_download_Test/Jeans24.jpg
DOWNLOADING: drive_download_Test/Jeans23.jpg
DOWNLOADING: drive_download_Test/Jeans22.jpg
DOWNLOADING: drive_download_Test/Jeans21.jpg
DOWNLOADING: drive_download_Test/Jeans20.jpg
DOWNLOADING: drive_download_Test/Jeans17.jpg
DOWNLOADING: drive_download_Test/Jeans18.jpg
DOWNLOADIN

In [71]:
test='/content/Test'

Data Preprocessing & Image Augmentation

In [ ]:
x_train = []
y_train = []
x_test = []
y_test = []

for feature, label in train:
  x_train.append(feature)
  y_train.append(label)

for feature, label in val:
  x_test.append(feature)
  y_test.append(label)

# Normalize the data
x_train = np.array(x_train) / 255
x_test = np.array(x_val) / 255

x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_val)

In [ ]:
#Model Building
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(224,224,3)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(2, activation="softmax"))

model.summary()

In [73]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()

In [ ]:
opt = optimizer(lr=0.000001)
model.compile(optimizer = opt , loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , metrics = ['accuracy'])

In [ ]:
history = model.fit(x_train,y_train,epochs = 500 , validation_data = (x_test, y_test))

In [ ]:
Evaluation the Result

In [ ]:
acc = history.history['accuracy']
test_acc = history.history['test_accuracy']
loss = history.history['loss']
test_loss = history.history['val_loss']

epochs_range = range(500)

In [ ]:
#Prediction
predictions = model.predict_classes(x_test)
predictions = predictions.reshape(1,-1)[0]
print(classification_report(y_test, predictions, target_names = ['Sarees (Class 0)','Jeans (Class 1)','Trousers (Class 2)']))